In [3]:
import plasmid
import importlib
import glob
import sys

In [5]:
x = plasmid.fileIO.read_fastx('merged/054b5d2a-9215-4ebf-9b85-8721d91a6309.fq.gz')

In [7]:
x[['name','sequence']]

,name,sequence
0,M03990:241:000000000-CYLNG:1:1101:17901:2039,GACGAACCGTCCAAACGTTATTCGGAATTATTGGGCTTAAAGGGTG...
1,M03990:241:000000000-CYLNG:1:1101:15414:2077,TACGTAGGGTGCAAGCGTTAATCGGAATTACTGGGCGTAAAGCGTG...
2,M03990:241:000000000-CYLNG:1:1101:11635:2095,TACGTAGGGTGCAAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCT...
3,M03990:241:000000000-CYLNG:1:1101:9323:2102,TACGGGGGGGGCAAGCGTTGTTCGGAATTACTGGGCGTAAAGGGCT...
4,M03990:241:000000000-CYLNG:1:1101:12109:2131,TACGTAGGGGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGAGCG...
...,...,...
82074,M03990:240:000000000-CY8DT:1:2119:23400:24891,AACGTAGGAGGCAAGCGTTATCCGGATTCACTGGGCGTAAAGCGCG...
82075,M03990:240:000000000-CY8DT:1:2119:11126:24934,TACGTAGGTGGCAAGCGTTGTCCGGATTTACTGGGCGTAAAGAGCG...
82076,M03990:240:000000000-CY8DT:1:2119:14099:24960,TACGTAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGCGTG...
82077,M03990:240:000000000-CY8DT:1:2119:18335:25149,TACGTAGGGCGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGCT...
